# **NLP2_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem.  """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Parameters:
l₁ = litres of component 1.
l₂ = litres of component 2.
m = mean litres utilised of both components = (l₁ + l₂) / 2.

Decision Variables:
Maximize Q = ln(l₁ * l₂) - ³√(l₁ + 2 * l₂).

Objective Function:
Maximize Q = ln(l₁ * l₂) - ³√(l₁ + 2 * l₂).

Constraints:
1. l₁ >= 0, l₂ >= 0 (litres used of each component should be non-negative).
2. (l₁ - m)² + (l₂ - m)² ≤ 10 (squared difference between the litres employed of each component and the mean litres utilised of both components is below ten).
3. l₁ + l₂ ≤ 80 (the mixture tank can only fit 80 litres).


### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
Markdown(response2.choices[0].message.content)

Here's a Python script using Pyomo to solve your optimization problem:

```python
from pyomo.environ import *
from math import log, pow

# Instantiate our problem
model = ConcreteModel()

# Pyomo has its log function and power function, but you can use the python math module if you want.
model.l1 = Var(domain=NonNegativeReals, initialize=0.5)
model.l2 = Var(domain=NonNegativeReals, initialize=0.5)

obj_expr = log(model.l1 * model.l2) - pow(model.l1 + 2 * model.l2, 1.0/3.0)
model.Q = Objective(expr=obj_expr, sense=maximize)

mean = (model.l1 + model.l2) / 2
model.con1 = Constraint(expr=(model.l1 - mean)**2 + (model.l2 - mean)**2 <= 10)
model.con2 = Constraint(expr=model.l1 + model.l2 <= 80)

# Solver
solver = SolverFactory('ipopt')
solver.solve(model)

print("Value of l1: ", model.l1())
print("Value of l2: ", model.l2())
print("Maximized Q: ", model.Q())
```

The ipopt solver is used because we have a nonlinear problem.

Sample data is used throughout this problem. Pyomo uses lazy evaluation so I didn't explicitly define the mean variable. If you want to get the mean after solving the problem just call:

```python
mean = (model.l1.value + model.l2.value) / 2
print("Mean: ", mean)
```

If you want to define the var `mean`, you should define it between the instantiation of `model = ConcreteModel()` and the constraints: 

```python
model.mean = Var(domain=NonNegativeReals, initialize=0.5)
```

And in the constraints and objective function, replace `mean` with `model.mean`. 

An initial guess for the decision variables `l1` and `l2` is given as `0.5` for the solver to start the process. This is a common practice in numerical optimization problems. 

Remember to install the `pyomo` and `ipopt` solver before running this script.

### **Run the code Generated by GPT4**

In [11]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.2 MB/s eta 0:00:00


In [12]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [13]:
from pyomo.environ import *
from math import log, pow

# Instantiate our problem
model = ConcreteModel()

# Pyomo has its log function and power function, but you can use the python math module if you want.
model.l1 = Var(domain=NonNegativeReals, initialize=0.5)
model.l2 = Var(domain=NonNegativeReals, initialize=0.5)

obj_expr = log(model.l1 * model.l2) - pow(model.l1 + 2 * model.l2, 1.0/3.0)
model.Q = Objective(expr=obj_expr, sense=maximize)

mean = (model.l1 + model.l2) / 2
model.con1 = Constraint(expr=(model.l1 - mean)**2 + (model.l2 - mean)**2 <= 10)
model.con2 = Constraint(expr=model.l1 + model.l2 <= 80)

# Solver
solver = SolverFactory('ipopt')
solver.solve(model)

print("Value of l1: ", model.l1())
print("Value of l2: ", model.l2())
print("Maximized Q: ", model.Q())

TypeError: Implicit conversion of Pyomo numeric value (l1*l2) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [22]:
from pyomo.environ import *
from pyomo.core.expr.current import log
from math import pow
# Instantiate our problem
model = ConcreteModel()

# Pyomo has its log function and power function, but you can use the python math module if you want.
model.l1 = Var(domain=NonNegativeReals)
model.l2 = Var(domain=NonNegativeReals)

obj_expr = log(model.l1 * model.l2) - (model.l1 + 2 * model.l2)**(1/3)
model.Q = Objective(expr=obj_expr, sense=maximize)

mean = (model.l1 + model.l2) / 2
model.con1 = Constraint(expr=(model.l1 - mean)**2 + (model.l2 - mean)**2 <= 10)
model.con2 = Constraint(expr=model.l1 + model.l2 <= 80)

# Solver
solver = SolverFactory('knitro')
solver.solve(model)

print("Value of l1: ", model.l1())
print("Value of l2: ", model.l2())
print("Maximized Q: ", model.Q())

Value of l1:  42.23606797533548
Value of l2:  37.76393202225084
Maximized Q:  2.4730339212529016
